# [Modules](https://docs.julialang.org/en/v1/manual/modules/)

Modules in Julia introduce new _**global**_ scopes, or to say; modules allow users to create top-level definitions (global variables).
1. One can have multiple files per module, and multiple modules per file.
1. One can include the same code in different modules. 

## Keyword `using` and `import`

_**If a name is qualified, then it can be accessed even if it is not exported.**_

### `using`

1. The [_**statement**_](https://github.com/lcy-seso/learning_notes/blob/master/programming_language/Glossary/basic_concepts.md#expression-and-statement-in-julia) `using Lib` means that a module called `Lib` will be _**available for resolving names**_ as needed.
    * When a global variable is encountered that has no definition in the current module, the system will _**search**_ for it among variables exported by `Lib` and import it if it is found there.
1. The [_**statement**_](https://github.com/lcy-seso/learning_notes/blob/master/programming_language/Glossary/basic_concepts.md#expression-and-statement-in-julia) `using Lib: thing1, thing2` brings just the identifiers `thing1` and `thing2` into scope from module `Lib`.
    * If `thing1` and `thing2` refer to functions, _**adding methods to them will not be allowed**_.
    
### `import`

1. The import _**keyword**_ only operates on a single name at a time.
1. Functions imported using `import` _**can be extended with new methods**_.
1. Imported variables are read-only.
    * Assigning to a global variable always affects a variable owned by the current module (does this means create a new local variable?), or else raises an error.

In [1]:
module MyTestModule1
export x, y, p, q

x = 1
y = 2
z = 3

p() = x
q() = y
r() = z

end

Main.MyTestModule1

In [2]:
using .MyTestModule1

@show x
@show y
@show p()
@show q()

@show MyTestModule1.z  # this is OK. If a name is qualified, then it can be accessed even if it is not exported.
@show z;   # only exported names are visible

x = 1
y = 2
p() = 1
q() = 2
MyTestModule1.z = 3


UndefVarError: UndefVarError: z not defined

In [3]:
@show MyTestModule1.r()  # this is OK.  If a name is qualified, then it can be accessed even if it is not exported.
@show r();  # only exported function is visible

MyTestModule1.r() = 3


UndefVarError: UndefVarError: r not defined

In [4]:
import .MyTestModule1

@show MyTestModule1.z
@show MyTestModule1.r();

MyTestModule1.z = 3
MyTestModule1.r() = 3


## Module: `Main`, `Core`, and `Base`

* _**`Main`**_: Julia starts with `Main` set as the current module.
    * Variables defined at the prompt go in `Main`.
    * `varinfo()` lists variables in `Main`.

* _**`Core`**_: Contains all identifiers considered "_**built in**_" to the language
    * `Core` includes part of the core language which are not libraries.
    * _**Every module**_ implicitly specifies `using Core`.

* _**`Base`**_: Contains basic functionality(the contents of [base/](https://github.com/JuliaLang/julia/tree/master/base)).
    * _**All modules**_ implicitly contain `using Base`, since this is needed in the vast majority of cases.
    
>In addition to `using Base`, modules also _**automatically contain**_ definitions of the `eval` and `include` functions: see [bare modules](https://docs.julialang.org/en/v1/manual/modules/#Default-top-level-definitions-and-bare-modules-1)

In [6]:
varinfo()

| name          |      size | summary |
|:------------- | ---------:|:------- |
| Base          |           | Module  |
| Core          |           | Module  |
| Main          |           | Module  |
| MyTestModule1 | 5.103 KiB | Module  |
| startupfile   |  51 bytes | String  |


## Load a module

Given the statement `using Foo`:
1. the system consults _**an internal table of top-level modules**_ to look for one named `Foo`.
1. If the module does not exist, the system attempts to `require(:Foo)`, which typically results in _**loading code from an installed package**_.


### Add search path for loading modules

When calling `require`, Julia searches for modules in directories specified by variable `LOAD_PATH` contains the directories.


`LOAD_PATH` can be extended using `push!`
    
```julia
push!(LOAD_PATH, "/Path/To/My/Module/")
```
        
1. Put this statement in the file `~/.julia/config/startup.jl` which extends `LOAD_PATH` on every Julia startup.
1. Define the environment variable `JULIA_LOAD_PATH`.

## `include` vs. `using` and `import`

see [this discussion](https://discourse.julialang.org/t/difference-between-using-and-include/1739/2)

* `include("file.jl")` evaluate the contents of `file.jl` _**in the current context**_(the global scope of the current module when the evaluation takes place).
* `include` _**is a function**_ that is typically used to load source interactively, or to combine files in packages that are broken into multiple source files.

>Another use for the textual “include” in other languages is _**reuse of code blocks**_ in local contexts (where the Julia `include` function may do surprising things). A Julian pattern for this would use macros. A nice example is the @def macro used inside the DifferentialEquations.jl ecosystem. There, one includes a file containing @def macro definitions, the definitions are evaluated right then, and the macros are used to paste in blocks of code at parsing time.